In [1]:
import json
import matplotlib.pyplot as plt
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_google_vertexai import ChatVertexAI
from langgraph.graph import END, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode
from google.cloud import bigquery

# Configurazioni iniziali
LOCATION = "us-central1"
LLM = "gemini-2.0-flash-001"

# Inizializza il client BigQuery
client = bigquery.Client()

# =======================
# 1. Query Agent: esegue query su BigQuery
# =======================
@tool
def query_bigquery(query_str: str) -> list[dict]:
    """
    Esegue una query SQL su BigQuery.
    Il parametro query_str deve essere una stringa JSON nel formato:
      {"query": "SELECT * FROM ..."}
    Restituisce i risultati della query come lista di dizionari.
    """
    try:
        data = json.loads(query_str)
        sql_query = data["query"]
        print(f"Esecuzione query: {sql_query}")
    except (json.JSONDecodeError, KeyError) as e:
        return [{"error": f"Formato di input non valido: {e}"}]
   
    try:
        query_job = client.query(sql_query)
        results = query_job.result()
        rows = [dict(row) for row in results]
        return rows
    except Exception as e:
        return [{"error": f"Errore nell'esecuzione della query: {str(e)}"}]

tools_query = [query_bigquery]

# Imposta il modello di linguaggio (lo stesso LLM verrà usato per gli agenti)
llm = ChatVertexAI(
    model=LLM,
    location=LOCATION,
    temperature=0,
    max_tokens=1024,
    streaming=True
).bind_tools(tools_query)

# Nodo per chiamare il modello e generare la query SQL
def call_model(state: MessagesState, config: RunnableConfig) -> dict[str, BaseMessage]:
    system_message = (
        "You are a helpful AI assistant who must provide a SQL BigQuery query "
        "to analyze data as the user requests. You can query data from: "
        "`qwiklabs-gcp-03-d68fba73ee2d.sales`. "
        "Return a JSON in the following format: {\"query\": \"SELECT * FROM ...\"} "
        "Do not use tool calls directly, just return the JSON. "
        "In BigQuery we have the following table (sales_online): "
        "Order_ID (INTEGER), Date (DATE), Customer (STRING), Product (STRING), "
        "Quantity (INTEGER), Price_per_unit (INTEGER), Total_Amount (INTEGER), "
        "Payment_Method (STRING)."
    )
    messages_with_system = [{"type": "system", "content": system_message}] + state["messages"]
    response = llm.invoke(messages_with_system, config)
    return {"messages": response}

def execute_query_from_response(state: MessagesState) -> dict:
    """
    Estrae ed esegue la query SQL dalla risposta del modello.
    Cerca un JSON formattato come {"query": "..."}.
    """
    last_message = state["messages"][-1]
    content = last_message.content
    json_query = None
    # Cerca il JSON racchiuso in ```json
    if "```json" in content:
        json_start = content.find("```json") + 7
        json_end = content.find("```", json_start)
        if json_end > json_start:
            json_query = content[json_start:json_end].strip()
    # Altrimenti cerca il JSON direttamente
    if not json_query and "{\"query\":" in content:
        json_start = content.find("{\"query\":")
        brace_count = 1
        for i in range(json_start + 1, len(content)):
            if content[i] == '{':
                brace_count += 1
            elif content[i] == '}':
                brace_count -= 1
                if brace_count == 0:
                    json_end = i + 1
                    break
        if brace_count == 0:
            json_query = content[json_start:json_end].strip()
    
    if json_query:
        result = query_bigquery(json_query)
        result_message = AIMessage(
            content=f"Ho eseguito la query e ecco i risultati:\n\n{result}"
        )
        return {"messages": [result_message]}
    
    error_message = AIMessage(
        content="Non sono riuscito a estrarre una query SQL valida dalla risposta."
    )
    return {"messages": [error_message]}

def should_continue(state: MessagesState) -> str:
    """
    Se l'ultimo messaggio contiene un comando JSON, prosegue con l'esecuzione della query,
    altrimenti termina.
    """
    last_message = state["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    if hasattr(last_message, 'content') and last_message.content:
        content = last_message.content
        if isinstance(content, str) and ("```json" in content or "{\"query\":" in content):
            return "execute_query"
    return END

workflow_query = StateGraph(MessagesState)
workflow_query.add_node("agent", call_model)
workflow_query.add_node("tools", ToolNode(tools_query))
workflow_query.add_node("execute_query", execute_query_from_response)
workflow_query.set_entry_point("agent")
workflow_query.add_conditional_edges("agent", should_continue)
workflow_query.add_edge("tools", "agent")
workflow_query.add_edge("execute_query", END)
query_agent = workflow_query.compile()


# =======================
# 2. Graph Agent: genera grafici a partire da un comando JSON
# =======================
@tool
def generate_graph(json_input: str) -> str:
    """
    Genera un grafico usando matplotlib.
    
    Il parametro json_input deve essere una stringa JSON nel formato:
      {"chart": "bar", "data": [{"label": "A", "value": 10}, {"label": "B", "value": 20}]}
    
    Salva il grafico come 'graph.png' e restituisce un messaggio di conferma.
    """
    try:
        data = json.loads(json_input)
        chart_type = data.get("chart")
        data_points = data.get("data")
        # Validazione dei dati
        if not isinstance(data_points, list) or not all(
            isinstance(dp, dict) and "label" in dp and "value" in dp for dp in data_points
        ):
            return "Errore: il campo 'data' deve essere una lista di oggetti con chiavi 'label' e 'value'."
    except Exception as e:
        return f"Errore nel parsing del JSON: {str(e)}"
    
    labels = [dp["label"] for dp in data_points]
    values = [dp["value"] for dp in data_points]
    plt.figure()
    if chart_type == "bar":
        plt.bar(labels, values)
    elif chart_type == "line":
        plt.plot(labels, values, marker='o')
    else:
        return "Errore: tipo di grafico non supportato. Usa 'bar' o 'line'."
    plt.xlabel("Label")
    plt.ylabel("Value")
    plt.title("Grafico generato")
    plt.tight_layout()
    plt.savefig("graph.png")
    plt.close()
    return "Grafico generato e salvato come 'graph.png'."

tools_graph = [generate_graph]

def call_graph_model(state: MessagesState, config: RunnableConfig) -> dict[str, BaseMessage]:
    """
    Chiede al modello di fornire un comando JSON per generare un grafico.
    Il comando JSON dovrà avere il formato:
      {"chart": "bar", "data": [{"label": "A", "value": 10}, ...]}
    """
    system_message = (
        "You are a helpful AI assistant who must provide a JSON command for generating a graph "
        "based on the provided data. Return a JSON in the following format: "
        "{\"chart\": \"bar\", \"data\": [{\"label\": \"A\", \"value\": 10}, ...]} "
        "Do not call any tools directly, just return the JSON."
    )
    messages_with_system = [{"type": "system", "content": system_message}] + state["messages"]
    response = llm.invoke(messages_with_system, config)
    return {"messages": response}

def execute_graph_from_response(state: MessagesState) -> dict:
    """
    Estrae il comando JSON per generare il grafico e lo esegue tramite il tool generate_graph.
    """
    last_message = state["messages"][-1]
    content = last_message.content
    json_command = None
    if "```json" in content:
        json_start = content.find("```json") + 7
        json_end = content.find("```", json_start)
        if json_end > json_start:
            json_command = content[json_start:json_end].strip()
    if not json_command and "{\"chart\":" in content:
        json_start = content.find("{\"chart\":")
        brace_count = 1
        for i in range(json_start + 1, len(content)):
            if content[i] == '{':
                brace_count += 1
            elif content[i] == '}':
                brace_count -= 1
                if brace_count == 0:
                    json_end = i + 1
                    break
        if brace_count == 0:
            json_command = content[json_start:json_end].strip()
    if json_command:
        result = generate_graph(json_command)
        result_message = AIMessage(content=f"Ho generato il grafico: {result}")
        return {"messages": [result_message]}
    error_message = AIMessage(content="Non sono riuscito a estrarre un comando JSON valido per generare il grafico.")
    return {"messages": [error_message]}

def should_continue_graph(state: MessagesState) -> str:
    """
    Determina se proseguire con il nodo per generare il grafico.
    """
    last_message = state["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "graph"
    if hasattr(last_message, 'content') and last_message.content:
        content = last_message.content
        if isinstance(content, str) and ("```json" in content or "{\"chart\":" in content):
            return "execute_graph"
    return END

workflow_graph = StateGraph(MessagesState)
workflow_graph.add_node("agent", call_graph_model)
workflow_graph.add_node("graph", ToolNode(tools_graph))
workflow_graph.add_node("execute_graph", execute_graph_from_response)
workflow_graph.set_entry_point("agent")
workflow_graph.add_conditional_edges("agent", should_continue_graph)
workflow_graph.add_edge("graph", "agent")
workflow_graph.add_edge("execute_graph", END)
graph_agent = workflow_graph.compile()


# =======================
# 3. Orchestrator Agent: coordina query e generazione grafici
# =======================
def orchestrator(state: MessagesState, config: RunnableConfig) -> dict[str, BaseMessage]:
    """
    Coordina l'esecuzione:
      1. Chiama il query_agent per ottenere dati da BigQuery.
      2. Con il risultato della query, costruisce un prompt per il graph_agent, che genera un grafico.
      3. Restituisce il risultato combinato.
    """
    # 1. Esegui il query_agent
    query_state = {"messages": state["messages"]}
    query_response = query_agent.invoke(query_state)
    
    # 2. Costruisci un prompt per il graph_agent includendo i dati ottenuti dalla query
    #    LLM del graph_agent dovrà trasformare i dati in un comando JSON per generare il grafico.
    graph_prompt = (
        "Genera un grafico a barre utilizzando i seguenti dati:\n"
        f"{query_response['messages'][-1].content}\n"
        "Il comando JSON deve avere il formato: "
        "{\"chart\": \"bar\", \"data\": [{\"label\": \"...\", \"value\": ...}, ...]}."
    )
    graph_state = {"messages": [{"type": "human", "content": graph_prompt}]}
    graph_response = graph_agent.invoke(graph_state)
    
    # 3. Combina i risultati e restituisci il messaggio finale
    combined_content = (
        f"Risultato query:\n{query_response['messages'][-1].content}\n\n"
        f"Risultato grafico:\n{graph_response['messages'][-1].content}"
    )
    final_message = AIMessage(content=combined_content)
    return {"messages": [final_message]}

workflow_orch = StateGraph(MessagesState)
workflow_orch.add_node("orchestrator", orchestrator)
workflow_orch.set_entry_point("orchestrator")
agent = workflow_orch.compile()

display(Image(agent.get_graph().draw_mermaid_png()))

ValidationError: 1 validation error for ChatVertexAI
  Value error, Unable to find your project. Please provide a project ID by:
- Passing a constructor argument
- Using vertexai.init()
- Setting project using 'gcloud config set project my-project'
- Setting a GCP environment variable
- To create a Google Cloud project, please follow guidance at https://developers.google.com/workspace/guides/create-project [type=value_error, input_value={'location': 'us-central1... 'default_metadata': ()}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to Building and Deploying an Agent with Agent Engine in Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/intro_agent_engine.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fagent-engine%2Fintro_agent_engine.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/intro_agent_engine.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/agent-engine/intro_agent_engine.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://goo.gle/4jeQzJW">
      <img width="32px" src="https://cdn.qwiklabs.com/assets/gcp_cloud-e3a77215f0b8bfa9b3f611c0d2208c7e8708ed31.svg" alt="Google Cloud logo"><br> Open in  Cloud Skills Boost
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/intro_agent_engine.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/intro_agent_engine.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/intro_agent_engine.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/intro_agent_engine.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/agent-engine/intro_agent_engine.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            


| | |
|-|-|
|Author(s) | [Kristopher Overholt](https://github.com/koverholt) |

## Overview

### Gemini

Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases.

### Function Calling in Gemini

[Function calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) lets developers create a description of a function in their code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

### Agent Engine in Vertex AI

[Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview) is a managed service that helps you to build and deploy an agent framework. It gives you the flexibility to choose how much reasoning you want to delegate to the LLM and how much you want to handle with customized code. You can define Python functions that get used as tools via [Gemini Function Calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling). Agent Engine integrates closely with the Python SDK for the Gemini model in Vertex AI, and it can manage prompts, agents, and examples in a modular way. Agent Engine is compatible with LangChain, LlamaIndex, or other Python frameworks.

<img width="60%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-engine-overview.png" alt="Agent Engine on Vertex AI" />

### Objectives

In this tutorial, you will learn how to build and deploy an agent (model, tools, and reasoning) using the Vertex AI SDK for Python.

You'll build and deploy an agent that uses the Gemini Pro model, Python functions as tools, and LangChain for orchestration.

You will complete the following tasks:

- Install the Vertex AI SDK for Python
- Use the Vertex AI SDK to build components of a simple agent
- Test your agent locally before deploying
- Deploy and test your agent on Vertex AI
- Customize each layer of your agent (model, tools, orchestration)

### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.


## Getting Started


### Install Vertex AI SDK for Python

Install the latest version of the Vertex AI SDK for Python as well as extra dependencies related to Agent Engine and LangChain:

In [1]:
%pip install --upgrade --quiet \
    "google-cloud-aiplatform[agent_engines,langchain]" \
    cloudpickle==3.0.0 \
    "pydantic>=2.10" \
    requests

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://[your-staging-bucket]"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Example: Build and deploy an agent

### Import libraries

In [5]:
from vertexai import agent_engines

### Define model

As you construct your agent from the bottom up, the first component deals with which generative model you want to use in your agent.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-1.png" alt="Components of an agent in Agent Engine on Vertex AI" />

Here you'll use the Gemini 1.5 Pro model:

In [6]:
model = "gemini-1.5-pro"

### Define Python functions (tools)

The second component of your agent includes tools and functions, which enable the generative model to interact with external systems, databases, document stores, and other APIs so that the model can get the most up-to-date information or take action with those systems.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-2.png" alt="Components of an agent in Agent Engine on Vertex AI" />

In this example, you'll define a function called `get_exchange_rate` that uses the `requests` library to retrieve real-time currency exchange information from an API:

In [7]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

Test the function with sample inputs to ensure that it's working as expected:

In [8]:
get_exchange_rate(currency_from="USD", currency_to="SEK")

{'amount': 1.0, 'base': 'USD', 'date': '2025-01-22', 'rates': {'SEK': 10.9806}}

### Define agent

The third component of your agent involves adding a reasoning layer, which helps your agent use the tools that you provided to help the end user achieve a higher-level goal.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-3.png" alt="Components of an agent in Agent Engine on Vertex AI" />

If you were to use Gemini and Function Calling on their own without a reasoning layer, you would need to handle the process of calling functions and APIs in your application code, and you would need to implement retries and additional logic to ensure that your function calling code is resilient to failures and malformed requests.

Here, you'll use the LangChain agent template provided in the Vertex AI SDK for Agent Engine, which brings together the model, tools, and reasoning that you've built up so far:

In [9]:
agent = agent_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

Now we can test the model and agent behavior to ensure that it's working as expected before we deploy it:

### Test your agent locally

With all of the core components of your agent in place, you can send a prompt to your agent using `.query` to test that it's working as expected, including the intermediate steps that the agent performed between the input prompt and the generated summary output. In the default mode, the agent processes your input and returns the **entire agent output in a single response when complete**:

In [10]:
agent.query(input="What's the exchange rate from US dollars to Swedish currency today?")

{'input': "What's the exchange rate from US dollars to Swedish currency today?",
 'output': 'The exchange rate from US dollars to Swedish krona is 1 USD to 10.9806 SEK. \n',
 'intermediate_steps': [[{'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'],
    'kwargs': {'tool': 'get_exchange_rate',
     'tool_input': {'currency_from': 'USD', 'currency_to': 'SEK'},
     'log': "\nInvoking: `get_exchange_rate` with `{'currency_from': 'USD', 'currency_to': 'SEK'}`\n\n\n",
     'type': 'AgentActionMessageLog',
     'message_log': [{'lc': 1,
       'type': 'constructor',
       'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'],
       'kwargs': {'content': '',
        'additional_kwargs': {'function_call': {'name': 'get_exchange_rate',
          'arguments': '{"currency_from": "USD", "currency_to": "SEK"}'}},
        'response_metadata': {'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH',
           'probability_label': 'NEGLIGI

In addition to the default query mode, the `.stream_query` method allows you to **see the agent's intermediate steps and final output from the chain**.

Instead of waiting for the agent to complete all sub-tasks, the agent sends back the response in **chunks as it's being generated**:

In [11]:
message_types = {"actions": "Action", "messages": "Message", "output": "Output"}
for chunk in agent.stream_query(
    input="What's the exchange rate from US dollars to Swedish currency today?"
):
    for key, label in message_types.items():
        if key in chunk:
            print("\n------\n")
            print(f"{label}:")
            print()
            print(chunk[key])


------

Action:

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'], 'kwargs': {'tool': 'get_exchange_rate', 'tool_input': {'currency_from': 'USD', 'currency_to': 'SEK'}, 'log': "\nInvoking: `get_exchange_rate` with `{'currency_from': 'USD', 'currency_to': 'SEK'}`\n\n\n", 'type': 'AgentActionMessageLog', 'message_log': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_exchange_rate', 'arguments': '{"currency_from": "USD", "currency_to": "SEK"}'}}, 'response_metadata': {'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.1416015625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.1240234375}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.2490234375, 'blocked': False, 's

This allows you to observe the agent's actions in real-time (such as function calls, and intermediate steps), which is helpful for debugging purposes or for providing real-time updates to the end user.

### Deploy your agent on Vertex AI

Now that you've specified a model, tools, and reasoning for your agent and tested it out, you're ready to deploy your agent as a remote service in Vertex AI!

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-4.png" alt="Components of an agent in Agent Engine on Vertex AI" />

You can re-define the agent to avoid any stateful information in the agent due to our testing in the previous cell:

In [12]:
agent = agent_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
)

Now you're ready to deploy your agent to Agent Engine in Vertex AI by calling `agent_engines.create()` along with:

1. The instance of your agent class
2. The Python packages and versions that your agent requires at runtime, similar to how you would define packages and versions in a `requirements.txt` file.

In [13]:
remote_agent = agent_engines.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
        "requests",
    ],
)

Using bucket your-bucket-name
Writing to gs://your-bucket-name/agent_engine/agent_engine.pkl
Writing to gs://your-bucket-name/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://your-bucket-name/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/your-project-number/locations/us-central1/reasoningEngines/3381833884248309760/operations/2331218534991921152
AgentEngine created. Resource name: projects/your-project-number/locations/us-central1/reasoningEngines/3381833884248309760
To use this AgentEngine in another session:
agent_engine = vertexai.agent_engines.get('projects/your-project-number/locations/us-central1/reasoningEngines/3381833884248309760')


Now you can send a prompt to your remote agent using `.query` to test that it's working as expected:

In [14]:
remote_agent.query(
    input="What's the exchange rate from US dollars to Swedish currency today?"
)

{'input': "What's the exchange rate from US dollars to Swedish currency today?",
 'output': 'The exchange rate from US dollars to Swedish krona is 1 USD to 10.9806 SEK. \n'}

Or you can stream the results back from the remote agent interactively using `.stream_query`:

In [15]:
message_types = {"actions": "Action", "messages": "Message", "output": "Output"}
for chunk in remote_agent.stream_query(
    input="What's the exchange rate from US dollars to Swedish currency today?"
):
    for key, label in message_types.items():
        if key in chunk:
            print("\n------\n")
            print(f"{label}:")
            print()
            print(chunk[key])


------

Action:

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'], 'kwargs': {'tool': 'get_exchange_rate', 'tool_input': {'currency_from': 'USD', 'currency_to': 'SEK'}, 'log': "\nInvoking: `get_exchange_rate` with `{'currency_from': 'USD', 'currency_to': 'SEK'}`\n\n\n", 'type': 'AgentActionMessageLog', 'message_log': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_exchange_rate', 'arguments': '{"currency_from": "USD", "currency_to": "SEK"}'}}, 'response_metadata': {'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_LOW'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGI

### Querying your deployed agent

You've now deployed your agent and can [interact with it in multiple ways](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/overview), both within this notebook and from other applications or environments. The primary methods for accessing your deployed agent are via the Python client library or through REST API calls. Here's an overview of both methods:

**Method 1: Reusing within this notebook or another Python environment**

You can directly reuse and query the `remote_agent` instance you created in this notebook.

Or, you can instantiate a new instance in another notebook or Python script. To do this, you'll need to retrieve your deployed agent's resource name that uniquely identifies your agent, which is a string that includes the project, location, and Agent Engine ID. You can retrieve it by running the following code in the notebook or environment where you created your agent:

In [16]:
remote_agent.resource_name

'projects/your-project-number/locations/us-central1/reasoningEngines/3381833884248309760'

Use the resource name to load the agent in your other notebook or Python script, then query the remote agent as usual:

In [17]:
# from vertexai import agent_engines

# AGENT_ENGINE_RESOURCE_NAME = "YOUR_AGENT_ENGINE_RESOURCE_NAME"  # Replace with the resource name of your deployed Agent Engine

# remote_agent = agent_engines.get(AGENT_ENGINE_RESOURCE_NAME)
# response = remote_agent.query(input=query)

**Method 2: Accessing from other environments via REST API**

Beyond the Python client library, your deployed Vertex AI agent can be [queried using REST API calls](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/overview), including:

- Python: You can use Python's `requests` library or similar tools to make HTTP calls to the Vertex AI REST API.
- cURL: A command-line tool, cURL allows you to send HTTP requests directly. This is useful for testing and debugging.
- Other Programming Languages: If you prefer a different language for your application, you can use its native HTTP client library to make REST API calls.

In summary, you have access to your deployed agent through the Python client library within Python environments, and more universally through its REST API via tools and programming languages of your choosing.

## Customizing your agent

The example above includes the minimal amount of configuration required for each component within the agent to help you get started.

But what if you want to swap to a different Gemini model version, change the generative model parameters or safety filters, or perform additional customizations to the agent? The following example shows some of the most common parameters that you'll want to customize in your agent. Agent Engine in Vertex AI works with [Gemini model versions that support Function Calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling) and LangChain agents.

### Model configuration

You'll start with customizations for the model component within your agent. Refer to the Vertex AI documentation for more information about configuration parameters related to [generative AI model versions](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versioning), [safety attributes](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes), and [model parameters](https://ai.google.dev/gemini-api/docs/models/gemini#model-attributes):

In [18]:
## Model variant and version
model = "gemini-1.5-pro"

## Model safety settings
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

## Model parameters
model_kwargs = {
    "temperature": 0.3,
    "max_output_tokens": 1000,
    "top_p": 0.95,
    "top_k": 40,
    "safety_settings": safety_settings,
}

### Agent configuration

You can also customize aspects of the agent component, including whether it should return all of the intermediate steps for a given input prompt and output response, maximum number of iterations, how to handle parsing errors, and the size of the sliding context window that the agent has visibility into:

In [19]:
# Agent parameters
agent_executor_kwargs = {
    "return_intermediate_steps": True,
    "max_iterations": 3,
    "handle_parsing_errors": False,
    "trim_intermediate_steps": -1,
}

Each component within your agent has additional parameters that can be customized or overridden, including the system instructions, preamble prompts, ability to manage chat sessions and history so that your agent maintains context between multi-turn queries.

Each component within your agent (model, tools, reasoning) is modular and composable so that you can customize your agent as little or as much as you want.

Refer to the [Agent Engine documentation in Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview) for more details on each agent component and the available options for customization.

## Cleaning up

After you've finished, it's a good practice to clean up your cloud resources. You can delete the deployed Agent Engine instance to avoid any unexpected charges on your Google Cloud account.

In [20]:
remote_agent.delete()